# Classification of California Census with TensorFlow 

Code Created by Luis Enrique Acevedo Galicia

Date: 2019-11-04

Here, I present a simple and easy way to create a classification with TensorFlow. In this case the data based on file census_data.csv. The targets are represented by incomes (>50k$ or <50k$) and the inputs are some live styles and live indicators.

# The Libraries

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import classification_report

# The data 

In [4]:
data = pd.read_csv('census_data.csv')
data.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Verifying that data is completed

In [5]:
data[pd.isnull(data['education_num'])]

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket


learning about this data set

In [6]:
data.describe()

,age,education_num,capital_gain,capital_loss,hours_per_week
count,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,10.080679,1077.648844,87.303830,40.437456
std,13.640433,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.000000,0.000000,0.000000,1.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000
50%,37.000000,10.000000,0.000000,0.000000,40.000000
75%,48.000000,12.000000,0.000000,0.000000,45.000000
max,90.000000,16.000000,99999.000000,4356.000000,99.000000


## Preprocessing Data

In [7]:
#Since there are some parameters in string formats, some preprocessing is neccesary.
#Let's begin with the income.
data['income_bracket'].unique()



array([' <=50K', ' >50K'], dtype=object)

In [10]:
#create a function to change from string to 1's and 0's

def label_to_num(lab):
    if lab==' <=50K':
        return 0
    else:
        return 1

In [12]:
#Transform the income
data['income_bracket'] = data['income_bracket'].apply(label_to_num)

## Split the data

In [13]:
Inputs_data = data.drop('income_bracket',axis=1)
Targets_data = data['income_bracket']
Inputs_train, Inputs_test, Targets_train, Targets_test = train_test_split(Inputs_data,Targets_data,test_size=0.3,random_state=101)

## Features Columns 

In [15]:
# first the categorical features.
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

#Numerical features

age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

#finally, the feature column
feature_C = [gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week]

## The model

In [18]:
#The input function
input_F = tf.estimator.inputs.pandas_input_fn(x=Inputs_train,y=Targets_train,batch_size=100,num_epochs=None,shuffle=True)

#The model will be a lienar classifier
LC_model = tf.estimator.LinearClassifier(feature_columns=feature_C)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpazdc1w_0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f82dcec3d30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [19]:
LC_model.train(input_fn=input_F, steps=5000)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpazdc1w_0/model.ckpt.
INFO:tensorflow:loss = 69.31474, step = 1
INFO:tensorflow:global_step/sec: 105.175
INFO:tensorflow:loss = 1289.2814, step = 101 (0.952 sec)
INFO:tensorflow:global_step/sec: 151.998
INFO:tensorflow:loss = 185.56314, step = 201 (0.658 sec)
INFO:tensorflow:global_step/sec: 165.026
INFO:tensorflow:loss = 58.219345, step = 301 (0.608 sec)
INFO:tensorflow:global_step/sec: 162.049
INFO:tensorflow:loss = 39.362526, step = 401 (0.615 sec)
INFO:

## Evaluation of the model

In [20]:
prediction_F = tf.estimator.inputs.pandas_input_fn(x=Inputs_test,batch_size=len(Inputs_test),shuffle=False)
predictions = list(LC_model.predict(input_fn=prediction_F))
predictions[0]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpazdc1w_0/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


{'logits': array([-1.0882471], dtype=float32),
 'logistic': array([0.2519485], dtype=float32),
 'probabilities': array([0.74805146, 0.2519485 ], dtype=float32),
 'class_ids': array([0]),
 'classes': array([b'0'], dtype=object)}

In [21]:
Prediction_final = []
for predict in predictions:
    Prediction_final.append(predict['class_ids'][0])

## Performance of the model

In [26]:
print(classification_report(Targets_test, Prediction_final))

              precision    recall  f1-score   support

           0       0.88      0.93      0.91      7436
           1       0.74      0.59      0.66      2333

   micro avg       0.85      0.85      0.85      9769
   macro avg       0.81      0.76      0.78      9769
weighted avg       0.85      0.85      0.85      9769

